In [1]:
import customtkinter as ctk
import tkinter as tk
from tkinter import messagebox
import os
import json
from collections import deque

# --- 1. STRUKTUR DATA (SAMA DENGAN FILE CLI ASLI) ---

# --- KONSTANTA FILE ---
# Menggunakan nama file yang berbeda agar tidak bentrok jika dijalankan di lingkungan yang sama
LIBRARY_FILE = 'syfoplay_library_gui.json' 
USERS_FILE = 'syfoplay_users_gui.json'

# 1. DATA MODELS / NODES
class Lagu:
    def __init__(self, id_lagu, judul, artis, genre, album, durasi, file_audio="simulated_path"):
        self.id_lagu = id_lagu
        self.judul = judul
        self.artis = artis
        self.genre = genre
        self.album = album
        self.durasi = durasi
        self.file_audio = file_audio
        self.next_lagu = None
        self.next_artis = None
        self.next_genre = None

class User:
    def __init__(self, email, username, password):
        self.email = email
        self.username = username
        self.password = password
        self.playlists = {}

    def to_dict(self):
        playlists_serial = {}
        for name, playlist_ll in self.playlists.items():
            playlists_serial[name] = playlist_ll.to_list_id()
        return {
            "email": self.email,
            "username": self.username,
            "password": self.password,
            "playlists": playlists_serial
        }

# 2. PLAYLIST (Singly Linked List)
class NodePlaylist:
    def __init__(self, lagu):
        self.lagu = lagu
        self.next = None

class PlaylistLinkedList:
    def __init__(self):
        self.head = None
        self.tail = None

    def tambah_lagu(self, lagu):
        new_node = NodePlaylist(lagu)
        if not self.head:
            self.head = new_node
            self.tail = new_node
        else:
            self.tail.next = new_node
            self.tail = new_node

    def hapus_lagu_berdasarkan_id(self, id_lagu):
        if not self.head:
            return False

        if self.head.lagu.id_lagu == id_lagu:
            self.head = self.head.next
            if not self.head:
                self.tail = None
            return True

        curr = self.head
        while curr.next:
            if curr.next.lagu.id_lagu == id_lagu:
                if curr.next == self.tail:
                    self.tail = curr
                curr.next = curr.next.next
                return True
            curr = curr.next
        return False
    
    def to_list_id(self):
        ids = []
        curr = self.head
        while curr:
            ids.append(curr.lagu.id_lagu)
            curr = curr.next
        return ids
    
    def to_list_lagu(self):
        """Mengembalikan list berisi objek Lagu"""
        lagu_list = []
        curr = self.head
        while curr:
            lagu_list.append(curr.lagu)
            curr = curr.next
        return lagu_list

# 3. LIBRARY (Singly Linked List)
class LibraryMultiLinkedList:
    def __init__(self):
        self.head = None
        self.lagu_counter = 1

    def _generate_id(self):
        id_baru = f"L{self.lagu_counter:03}"
        self.lagu_counter += 1
        return id_baru

    def tambah_lagu(self, judul, artis, genre, album, durasi):
        id_baru = self._generate_id()
        lagu_baru = Lagu(id_baru, judul, artis, genre, album, durasi)
        lagu_baru.next_lagu = self.head
        self.head = lagu_baru
        return lagu_baru

    def get_all_lagu(self):
        lagu_list = []
        curr = self.head
        while curr:
            lagu_list.append(curr)
            curr = curr.next_lagu
        lagu_list.sort(key=lambda l: l.id_lagu) # Sort by ID for consistent display
        return lagu_list

    def cari_lagu(self, kunci, nilai):
        hasil = []
        curr = self.head
        nilai_lower = nilai.lower()
        while curr:
            if kunci == "judul" and nilai_lower in curr.judul.lower():
                hasil.append(curr)
            elif kunci == "artis" and nilai_lower in curr.artis.lower():
                hasil.append(curr)
            elif kunci == "genre" and nilai_lower in curr.genre.lower():
                hasil.append(curr)
            elif kunci == "album" and nilai_lower in curr.album.lower():
                hasil.append(curr)
            curr = curr.next_lagu
        return hasil

    def cari_lagu_berdasarkan_id(self, id_lagu):
        curr = self.head
        while curr:
            if curr.id_lagu == id_lagu:
                return curr
            curr = curr.next_lagu
        return None

    def hapus_lagu_dan_update_playlist(self, id_lagu, all_users):
        if not self.head:
            return False

        lagu_dihapus = None
        if self.head.lagu.id_lagu == id_lagu:
            lagu_dihapus = self.head
            self.head = self.head.next_lagu
        else:
            curr = self.head
            while curr.next_lagu:
                if curr.next_lagu.lagu.id_lagu == id_lagu:
                    lagu_dihapus = curr.next_lagu
                    curr.next_lagu = curr.next_lagu.next_lagu
                    break
                curr = curr.next_lagu

        if not lagu_dihapus:
            return False

        for username, user in all_users.items():
            for nama_playlist, playlist_ll in user.playlists.items():
                playlist_ll.hapus_lagu_berdasarkan_id(id_lagu)
        return True

# 4. Playback queue (deque) dan history (stack)
class PlaybackQueue:
    def __init__(self):
        self._items = deque()
    def enqueue(self, lagu):
        self._items.append(lagu)
    def dequeue(self):
        if self._items:
            return self._items.popleft()
        return None
    def is_empty(self):
        return len(self._items) == 0
    def __len__(self):
        return len(self._items)

class PlaybackHistory:
    def __init__(self):
        self._stack = []
    def push(self, lagu):
        self._stack.append(lagu)
    def pop(self):
        if self._stack:
            return self._stack.pop()
        return None
    def is_empty(self):
        return len(self._stack) == 0
    def __len__(self):
        return len(self._stack)


# 5. APLIKASI UTAMA (BACKEND)
class SyfoplayAppLogic:
    def __init__(self):
        self.LIBRARY_FILE = LIBRARY_FILE
        self.USERS_FILE = USERS_FILE
        self.library = LibraryMultiLinkedList()
        self.users = {}
        self.admin_id = "admin"
        self.admin_pass = "admin123"
        self.current_user = None
        self.current_playing = None
        self.playback_queue = PlaybackQueue()
        self.history = PlaybackHistory()
        
        self._load_data()
        
    # --- JSON PERSISTENCE METHODS ---
    def _load_data(self):
        # 1. Load Library
        try:
            with open(self.LIBRARY_FILE, 'r') as f:
                data_lib = json.load(f)
            
            self.library.lagu_counter = data_lib.get("lagu_counter", 1)
            
            head_node = None
            # Memuat lagu dari list, harus dari yang terakhir ditambahkan agar urutan LL benar
            for item in reversed(data_lib.get("library", [])): 
                lagu = Lagu(**item)
                lagu.next_lagu = head_node
                head_node = lagu
            self.library.head = head_node

        except (FileNotFoundError, json.JSONDecodeError):
            print("[INFO] File Library tidak ditemukan/rusak. Menggunakan library kosong.")

        # 2. Load Users
        try:
            with open(self.USERS_FILE, 'r') as f:
                data_users = json.load(f)

            for username, user_data in data_users.items():
                user = User(user_data["email"], user_data["username"], user_data["password"])
                
                for name, list_id_lagu in user_data.get("playlists", {}).items():
                    playlist_ll = PlaylistLinkedList()
                    for id_lagu in list_id_lagu:
                        lagu_ref = self.library.cari_lagu_berdasarkan_id(id_lagu)
                        if lagu_ref:
                            playlist_ll.tambah_lagu(lagu_ref)
                    user.playlists[name] = playlist_ll
                
                self.users[username] = user

        except (FileNotFoundError, json.JSONDecodeError):
            print("[INFO] File Users tidak ditemukan/rusak. Menggunakan user kosong.")
            
    def _save_library(self):
        data_lib = {
            "lagu_counter": self.library.lagu_counter,
            "library": []
        }
        curr = self.library.head
        
        while curr:
            lagu_dict = vars(curr).copy()
            lagu_dict.pop('next_lagu', None)
            lagu_dict.pop('next_artis', None)
            lagu_dict.pop('next_genre', None)
            data_lib["library"].append(lagu_dict)
            curr = curr.next_lagu
        
        # Simpan dalam urutan ID yang konsisten
        data_lib["library"].sort(key=lambda x: x['id_lagu']) 
        
        try:
            with open(self.LIBRARY_FILE, 'w') as f:
                json.dump(data_lib, f, indent=2)
        except Exception as e:
            print(f"[ERROR] Gagal menyimpan library: {e}")

    def _save_users(self):
        data_users = {}
        for username, user in self.users.items():
            data_users[username] = user.to_dict()

        try:
            with open(self.USERS_FILE, 'w') as f:
                json.dump(data_users, f, indent=2)
        except Exception as e:
            print(f"[ERROR] Gagal menyimpan users: {e}")

    # --- PLAYBACK LOGIC ---
    def play_lagu(self, lagu):
        if self.current_playing:
            self.history.push(self.current_playing)
        self.current_playing = lagu

    def next_queue(self):
        lagu_next = self.playback_queue.dequeue()
        if lagu_next:
            self.play_lagu(lagu_next)
            return True
        else:
            self.current_playing = None
            return False

    def prev_history(self):
        lagu_prev = self.history.pop()
        if lagu_prev:
            if self.current_playing:
                # Simpan lagu yang sedang diputar ke queue, karena kita mundur
                self.playback_queue._items.appendleft(self.current_playing) 
            self.current_playing = lagu_prev
            return True
        else:
            return False

# --- 2. IMPLEMENTASI GUI DENGAN CUSTOMTKINTER ---

ctk.set_appearance_mode("Dark")
ctk.set_default_color_theme("blue")

class SyfoplayGUI(ctk.CTk):
    def __init__(self, logic):
        super().__init__()
        self.logic = logic

        self.title("Syfoplay (GUI)")
        self.geometry("800x600")
        self.min_width = 800
        self.min_height = 600
        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)

        # Container Frame (Menampung semua halaman)
        self.container = ctk.CTkFrame(self)
        self.container.grid(row=0, column=0, sticky="nsew")
        self.container.grid_rowconfigure(0, weight=1)
        self.container.grid_columnconfigure(0, weight=1)

        # FIX UTAMA: Playback Bar harus diinisialisasi sebelum self.show_frame()
        self.playback_bar = PlaybackBar(self, self.logic)
        self.playback_bar.grid(row=1, column=0, sticky="ew")
        
        self.frames = {}
        self.create_frames()
        self.show_frame("LoginPage") 
        
        # Binding untuk menutup window
        self.protocol("WM_DELETE_WINDOW", self.on_closing)

    def on_closing(self):
        """Memastikan data tersimpan saat aplikasi ditutup."""
        try:
            self.logic._save_library()
            self.logic._save_users()
            print("Data tersimpan sebelum keluar.")
        except Exception as e:
            print(f"Gagal menyimpan data saat keluar: {e}")
        self.destroy()

    def create_frames(self):
        # Inisialisasi semua frame/halaman
        for F in (LoginPage, UserSignupPage, AdminDashboardPage, UserDashboardPage, UserPlaylistMenuPage, AdminAddUpdateSongPage):
            page_name = F.__name__
            frame = F(parent=self.container, controller=self, logic=self.logic)
            self.frames[page_name] = frame
            frame.grid(row=0, column=0, sticky="nsew")

    def show_frame(self, page_name, *args, **kwargs):
        """Menampilkan halaman yang diminta."""
        frame = self.frames[page_name]
        
        # Panggil metode refresh (jika ada) sebelum ditampilkan
        if hasattr(frame, 'refresh_content'):
            frame.refresh_content(*args, **kwargs)

        frame.tkraise()
        # Panggilan update playback info setelah frame ditampilkan
        self.playback_bar.update_playback_info() 
        
    def get_frame(self, page_name):
        return self.frames[page_name]

# --- PLAYBACK BAR DI BAWAH ---
class PlaybackBar(ctk.CTkFrame):
    def __init__(self, master, logic):
        super().__init__(master, height=60, fg_color="#181818") # Warna gelap khas media player
        self.logic = logic
        
        self.grid_columnconfigure(0, weight=1)
        self.grid_columnconfigure(1, weight=1)
        self.grid_columnconfigure(2, weight=1)

        # 1. Info Lagu
        self.song_info_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.song_info_frame.grid(row=0, column=0, padx=10, pady=5, sticky="w")
        self.song_info_frame.grid_columnconfigure(0, weight=1)
        
        self.title_label = ctk.CTkLabel(self.song_info_frame, text="[Tidak Ada Lagu]", font=ctk.CTkFont(weight="bold"))
        self.title_label.grid(row=0, column=0, sticky="w")
        self.artist_label = ctk.CTkLabel(self.song_info_frame, text="Oleh: -")
        self.artist_label.grid(row=1, column=0, sticky="w")
        
        # 2. Kontrol Playback
        self.controls_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.controls_frame.grid(row=0, column=1, padx=5, pady=5, sticky="nsew")
        self.controls_frame.grid_columnconfigure((0, 1, 2), weight=1)
        
        self.prev_button = ctk.CTkButton(self.controls_frame, text="⏮️", width=30, command=self.prev_song)
        self.prev_button.grid(row=0, column=0, padx=5, pady=5)
        self.play_pause_button = ctk.CTkButton(self.controls_frame, text="▶️", width=30, command=self.toggle_play_pause)
        self.play_pause_button.grid(row=0, column=1, padx=5, pady=5)
        self.next_button = ctk.CTkButton(self.controls_frame, text="⏭️", width=30, command=self.next_song)
        self.next_button.grid(row=0, column=2, padx=5, pady=5)
        
        # 3. Progress Bar (Simulasi)
        self.progress_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.progress_frame.grid(row=0, column=2, padx=10, pady=5, sticky="e")
        
        self.progress_bar = ctk.CTkProgressBar(self.progress_frame, width=150)
        self.progress_bar.set(0)
        self.progress_bar.pack(pady=5)
        self.time_label = ctk.CTkLabel(self.progress_frame, text="0:00 / 0:00")
        self.time_label.pack()
        
    def update_playback_info(self):
        """Memperbarui info lagu yang sedang diputar."""
        if self.logic.current_playing:
            lagu = self.logic.current_playing
            self.title_label.configure(text=lagu.judul)
            self.artist_label.configure(text=f"Oleh: {lagu.artis} ({lagu.durasi})")
            self.play_pause_button.configure(text="⏸️") # Anggap lagu sedang 'diputar'
            self.time_label.configure(text=f"0:00 / {lagu.durasi}")
            self.progress_bar.set(0.2) # Simulasi progress
        else:
            self.title_label.configure(text="[Tidak Ada Lagu]")
            self.artist_label.configure(text="Oleh: -")
            self.play_pause_button.configure(text="▶️")
            self.time_label.configure(text="0:00 / 0:00")
            self.progress_bar.set(0)

    def toggle_play_pause(self):
        """Simulasi Play/Pause. Dalam implementasi nyata, ini akan lebih kompleks."""
        if self.logic.current_playing:
            # Di sini bisa ditambahkan logika pause/resume
            messagebox.showinfo("Playback", f"Simulasi Play/Pause untuk: {self.logic.current_playing.judul}")
            # Saat ini hanya switch icon
            if self.play_pause_button.cget("text") == "⏸️":
                 self.play_pause_button.configure(text="▶️")
            else:
                 self.play_pause_button.configure(text="⏸️")
        else:
             messagebox.showinfo("Playback", "Tidak ada lagu yang sedang diputar.")

    def next_song(self):
        if self.logic.next_queue():
            # Refresh tampilan untuk update state
            if self.master.frames:
                current_page_name = next(name for name, frame in self.master.frames.items() if frame.winfo_ismapped())
                self.master.show_frame(current_page_name) 
            messagebox.showinfo("Playback", f"Memutar lagu berikutnya: {self.logic.current_playing.judul}")
        else:
            messagebox.showinfo("Playback", "Queue kosong.")
        self.update_playback_info()

    def prev_song(self):
        if self.logic.prev_history():
            # Refresh tampilan untuk update state
            if self.master.frames:
                current_page_name = next(name for name, frame in self.master.frames.items() if frame.winfo_ismapped())
                self.master.show_frame(current_page_name)
            messagebox.showinfo("Playback", f"Memutar lagu sebelumnya: {self.logic.current_playing.judul}")
        else:
            messagebox.showinfo("Playback", "History kosong.")
        self.update_playback_info()


# --- HALAMAN-HALAMAN APLIKASI ---

# Halaman Login/Menu Utama (Admin Login)
class LoginPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent, fg_color="#204060") # Warna latar belakang sesuai visualisasi
        self.controller = controller
        self.logic = logic
        
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)
        
        # Frame Login Admin
        login_frame = ctk.CTkFrame(self, width=350, height=450, corner_radius=10)
        login_frame.grid(row=0, column=0, padx=20, pady=20)
        
        login_label = ctk.CTkLabel(login_frame, text="SYFOPLAY", font=ctk.CTkFont(size=24, weight="bold"))
        login_label.pack(pady=20, padx=60)
        
        ctk.CTkLabel(login_frame, text="Login Admin", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=(10, 5))
        
        self.id_admin = ctk.CTkEntry(login_frame, width=200, placeholder_text="ID Admin")
        self.id_admin.pack(pady=5)
        self.pass_admin = ctk.CTkEntry(login_frame, width=200, placeholder_text="Password Admin", show="*")
        self.pass_admin.pack(pady=5)
        
        login_button = ctk.CTkButton(login_frame, text="MASUK", command=self.admin_login)
        login_button.pack(pady=10)

        ctk.CTkLabel(login_frame, text="--- ATAU ---").pack(pady=5)
        
        user_button = ctk.CTkButton(login_frame, text="MASUK SEBAGAI USER", command=lambda: controller.show_frame("UserSignupPage"))
        user_button.pack(pady=10)

        # Tombol Keluar
        exit_button = ctk.CTkButton(login_frame, text="KELUAR APLIKASI", fg_color="red", hover_color="#800000", command=controller.on_closing)
        exit_button.pack(pady=10)

    def admin_login(self):
        admin_id = self.id_admin.get()
        admin_pass = self.pass_admin.get()
        
        if admin_id == self.logic.admin_id and admin_pass == self.logic.admin_pass:
            messagebox.showinfo("Login Berhasil", "Selamat bekerja, Admin!")
            self.id_admin.delete(0, tk.END)
            self.pass_admin.delete(0, tk.END)
            self.controller.show_frame("AdminDashboardPage")
        else:
            messagebox.showerror("Login Gagal", "ID atau Password Admin tidak sesuai.")


# Halaman Sign Up / Login User
class UserSignupPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent, fg_color="#204060")
        self.controller = controller
        self.logic = logic
        
        self.grid_columnconfigure(0, weight=1)
        self.grid_rowconfigure(0, weight=1)
        
        self.current_mode = tk.StringVar(value="signup") # Default ke Sign Up
        
        self.main_frame = ctk.CTkFrame(self, width=400, height=500, corner_radius=10)
        self.main_frame.grid(row=0, column=0, padx=20, pady=20)
        
        # Header
        ctk.CTkLabel(self.main_frame, text="SYFOPLAY", font=ctk.CTkFont(size=24, weight="bold")).pack(pady=20)
        
        # Tombol Switch (Login/Signup)
        self.switch_frame = ctk.CTkFrame(self.main_frame, fg_color="transparent")
        self.switch_frame.pack(pady=(0, 10))
        
        # Inisialisasi Tombol
        self.login_switch = ctk.CTkButton(self.switch_frame, text="Login", command=lambda: self.switch_mode("login"))
        self.login_switch.grid(row=0, column=0, padx=5)
        self.signup_switch = ctk.CTkButton(self.switch_frame, text="Sign Up", command=lambda: self.switch_mode("signup"))
        self.signup_switch.grid(row=0, column=1, padx=5)

        # FIX UTAMA: Tangkap warna default setelah inisialisasi tombol
        self._default_fg_color = self.login_switch.cget("fg_color")
        self._default_hover_color = self.login_switch.cget("hover_color")

        # Area Konten (diisi oleh fungsi switch_mode)
        self.content_frame = ctk.CTkFrame(self.main_frame, fg_color="transparent")
        self.content_frame.pack(padx=20, pady=10, fill="x")
        
        # Tombol Kembali
        ctk.CTkButton(self.main_frame, text="< KEMBALI", command=lambda: controller.show_frame("LoginPage")).pack(pady=10)
        
        self.switch_mode("signup") # Tampilkan Sign Up pertama kali

    def switch_mode(self, mode):
        """Mengganti tampilan antara Sign Up dan Login."""
        if self.current_mode.get() == mode:
            return

        # Bersihkan konten frame
        for widget in self.content_frame.winfo_children():
            widget.destroy()

        self.current_mode.set(mode)
        
        if mode == "signup":
            # Sorot tombol Sign Up (aktif: hijau, non-aktif: default yang sudah disimpan)
            self.login_switch.configure(fg_color=self._default_fg_color, hover_color=self._default_hover_color)
            self.signup_switch.configure(fg_color="green", hover_color="#006400")
            self.create_signup_fields()
        else: # mode == "login"
            # Sorot tombol Login (aktif: hijau, non-aktif: default yang sudah disimpan)
            self.signup_switch.configure(fg_color=self._default_fg_color, hover_color=self._default_hover_color)
            self.login_switch.configure(fg_color="green", hover_color="#006400")
            self.create_login_fields()

    def create_signup_fields(self):
        ctk.CTkLabel(self.content_frame, text="--- Sign Up ---", font=ctk.CTkFont(size=16, weight="bold")).pack(pady=5)
        
        self.signup_email = ctk.CTkEntry(self.content_frame, width=250, placeholder_text="Email")
        self.signup_email.pack(pady=5)
        self.signup_username = ctk.CTkEntry(self.content_frame, width=250, placeholder_text="Username")
        self.signup_username.pack(pady=5)
        self.signup_password = ctk.CTkEntry(self.content_frame, width=250, placeholder_text="Password", show="*")
        self.signup_password.pack(pady=5)
        self.signup_verif_pass = ctk.CTkEntry(self.content_frame, width=250, placeholder_text="Verifikasi Password", show="*")
        self.signup_verif_pass.pack(pady=5)
        
        ctk.CTkButton(self.content_frame, text="DAFTAR", command=self.handle_signup).pack(pady=10)

    def create_login_fields(self):
        ctk.CTkLabel(self.content_frame, text="--- Login User ---", font=ctk.CTkFont(size=16, weight="bold")).pack(pady=5)

        self.login_username = ctk.CTkEntry(self.content_frame, width=250, placeholder_text="Username")
        self.login_username.pack(pady=5)
        self.login_password = ctk.CTkEntry(self.content_frame, width=250, placeholder_text="Password", show="*")
        self.login_password.pack(pady=5)

        ctk.CTkButton(self.content_frame, text="MASUK", command=self.handle_login).pack(pady=10)
        
    def handle_signup(self):
        email = self.signup_email.get().strip()
        username = self.signup_username.get().strip()
        password = self.signup_password.get().strip()
        verif_pass = self.signup_verif_pass.get().strip()

        if not (email and username and password and verif_pass):
            messagebox.showerror("Sign Up Gagal", "Semua field harus diisi.")
            return
            
        if password != verif_pass:
            messagebox.showerror("Sign Up Gagal", "Verifikasi Password tidak sesuai!")
            return
            
        if username in self.logic.users:
            messagebox.showerror("Sign Up Gagal", "Username sudah terdaftar!")
            return
        
        # Proses Pendaftaran
        self.logic.users[username] = User(email, username, password)
        self.logic._save_users()
        messagebox.showinfo("Sign Up Berhasil", f"Akun {username} berhasil dibuat! Silakan Login.")
        self.switch_mode("login") # Langsung pindah ke mode login

    def handle_login(self):
        username = self.login_username.get().strip()
        password = self.login_password.get().strip()
        
        # Validasi field kosong secara eksplisit
        if not (username and password):
             messagebox.showerror("Login Gagal", "Username dan Password harus diisi.")
             return
        
        user = self.logic.users.get(username)
        if user and user.password == password:
            self.logic.current_user = user
            messagebox.showinfo("Login Berhasil", f"Selamat datang, {username}!")
            self.login_username.delete(0, tk.END)
            self.login_password.delete(0, tk.END)
            self.controller.show_frame("UserDashboardPage")
        else:
            messagebox.showerror("Login Gagal", "Username atau Password salah.")


# Dashboard Admin
class AdminDashboardPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        
        self.grid_rowconfigure(1, weight=1) # Konten utama
        self.grid_columnconfigure(0, weight=1)
        
        # Header Admin
        header_frame = ctk.CTkFrame(self, fg_color="#336699")
        header_frame.grid(row=0, column=0, sticky="ew", padx=0, pady=0)
        header_frame.grid_columnconfigure(0, weight=1)
        
        ctk.CTkLabel(header_frame, text="HAI ADMIN, SELAMAT BEKERJA! 👋", font=ctk.CTkFont(size=20, weight="bold")).grid(row=0, column=0, padx=10, pady=10, sticky="w")
        ctk.CTkButton(header_frame, text="KELUAR", command=self.logout).grid(row=0, column=1, padx=10, pady=10, sticky="e")
        
        # Konten Utama
        self.content_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.content_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=10)
        self.content_frame.grid_columnconfigure(0, weight=3) # Daftar Lagu (Kiri)
        self.content_frame.grid_columnconfigure(1, weight=1) # Menu Aksi (Kanan)
        self.content_frame.grid_rowconfigure(0, weight=1) 
        
        # Container untuk Daftar Lagu dan Label
        list_container = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        list_container.grid(row=0, column=0, sticky="nsew", padx=5, pady=5)
        list_container.grid_rowconfigure(1, weight=1) # Row 1 untuk scrollable frame
        list_container.grid_columnconfigure(0, weight=1)

        # 1. Daftar Lagu - Label dan Scrollable Frame
        ctk.CTkLabel(list_container, text="Daftar Isi Database Lagu", font=ctk.CTkFont(weight="bold")).grid(row=0, column=0, sticky="w", padx=5, pady=(0, 5))
        self.list_frame = ctk.CTkScrollableFrame(list_container) 
        self.list_frame.grid(row=1, column=0, sticky="nsew") 

        # 2. Menu Aksi 
        self.action_frame = ctk.CTkFrame(self.content_frame)
        self.action_frame.grid(row=0, column=1, sticky="n", padx=5, pady=5)
        
        # Manual Label untuk judul frame Aksi Admin
        ctk.CTkLabel(self.action_frame, text="Aksi Admin", font=ctk.CTkFont(weight="bold")).pack(pady=(10, 5)) 
        
        ctk.CTkButton(self.action_frame, text="➕ TAMBAH LAGU", command=lambda: controller.show_frame("AdminAddUpdateSongPage", mode="tambah")).pack(pady=10, padx=20, fill="x")
        self.update_button = ctk.CTkButton(self.action_frame, text="✏️ UPDATE LAGU", command=self.start_update_lagu, state=tk.DISABLED)
        self.update_button.pack(pady=10, padx=20, fill="x")
        self.delete_button = ctk.CTkButton(self.action_frame, text="❌ HAPUS LAGU", fg_color="red", hover_color="#800000", command=self.hapus_lagu, state=tk.DISABLED)
        self.delete_button.pack(pady=10, padx=20, fill="x")
        
        # Pencarian
        ctk.CTkLabel(self.action_frame, text="Pencarian").pack(pady=(20, 5))
        self.search_entry = ctk.CTkEntry(self.action_frame, placeholder_text="Ketik Judul/Artis/Genre/Album")
        self.search_entry.pack(pady=5, padx=20, fill="x")
        self.search_var = tk.StringVar(value="judul")
        search_options = ctk.CTkOptionMenu(self.action_frame, values=["judul", "artis", "genre", "album"], variable=self.search_var)
        search_options.pack(pady=5, padx=20, fill="x")
        ctk.CTkButton(self.action_frame, text="CARI", command=self.cari_lagu).pack(pady=5, padx=20, fill="x")
        ctk.CTkButton(self.action_frame, text="RESET", command=lambda: self.refresh_content(reset_search=True)).pack(pady=5, padx=20, fill="x")


    def refresh_content(self, search_results=None, reset_search=False):
        """Memuat ulang daftar lagu."""
        self.selected_song_id = None
        if hasattr(self, 'selected_row') and self.selected_row:
            self.selected_row.configure(fg_color="transparent")
            self.selected_row = None
        self.update_button.configure(state=tk.DISABLED)
        self.delete_button.configure(state=tk.DISABLED)
        
        if reset_search:
            self.search_entry.delete(0, tk.END)

        # Hapus konten lama di list_frame
        for widget in self.list_frame.winfo_children():
            widget.destroy()

        # Header Tabel
        headers = ["No", "ID", "Judul", "Artis", "Genre", "Album", "Durasi", "Play"]
        for i, text in enumerate(headers):
            # Mengatur grid column configure di dalam list_frame (Scrollable)
            self.list_frame.grid_columnconfigure(i, weight=1)
            ctk.CTkLabel(self.list_frame, text=text, font=ctk.CTkFont(weight="bold")).grid(row=0, column=i, sticky="w", padx=5)

        # Data Lagu
        lagu_list = search_results if search_results is not None else self.logic.library.get_all_lagu()

        if not lagu_list:
            ctk.CTkLabel(self.list_frame, text="[Library kosong.]", anchor="w").grid(row=1, column=0, columnspan=8, padx=5, pady=10)
            return

        for i, lagu in enumerate(lagu_list, 1):
            row_frame = ctk.CTkFrame(self.list_frame, fg_color="transparent")
            row_frame.grid(row=i, column=0, columnspan=8, sticky="ew")
            row_frame.grid_columnconfigure((0, 1, 2, 3, 4, 5, 6, 7), weight=1)

            # Data Kolom
            ctk.CTkLabel(row_frame, text=f"{i}").grid(row=0, column=0, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.id_lagu).grid(row=0, column=1, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.judul[:20]).grid(row=0, column=2, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.artis[:15]).grid(row=0, column=3, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.genre[:10]).grid(row=0, column=4, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.album[:13]).grid(row=0, column=5, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.durasi).grid(row=0, column=6, sticky="w", padx=5)
            
            # Tombol Play/Select
            play_button = ctk.CTkButton(row_frame, text="▶️", width=30, command=lambda l=lagu: self.play_song_admin(l))
            play_button.grid(row=0, column=7, sticky="w", padx=5)

            # Bind klik untuk memilih baris
            row_frame.bind("<Button-1>", lambda event, l=lagu, rf=row_frame: self.select_song(l.id_lagu, rf))
            for widget in row_frame.winfo_children():
                widget.bind("<Button-1>", lambda event, l=lagu, rf=row_frame: self.select_song(l.id_lagu, rf))

    def select_song(self, song_id, row_frame):
        """Menandai baris yang dipilih dan mengaktifkan tombol aksi."""
        if hasattr(self, 'selected_row') and self.selected_row:
            self.selected_row.configure(fg_color="transparent") # Reset warna baris sebelumnya

        self.selected_song_id = song_id
        self.selected_row = row_frame
        self.selected_row.configure(fg_color="#454545") # Warna sorotan
        
        self.update_button.configure(state=tk.NORMAL)
        self.delete_button.configure(state=tk.NORMAL)
        
    def start_update_lagu(self):
        """Memulai proses update lagu."""
        if self.selected_song_id:
            lagu_node = self.logic.library.cari_lagu_berdasarkan_id(self.selected_song_id)
            if lagu_node:
                self.controller.show_frame("AdminAddUpdateSongPage", mode="update", lagu_node=lagu_node)
            else:
                messagebox.showerror("Error", "Lagu tidak ditemukan.")
        else:
            messagebox.showwarning("Perhatian", "Pilih lagu yang akan di-update.")

    def hapus_lagu(self):
        """Menghapus lagu yang dipilih."""
        if self.selected_song_id:
            response = messagebox.askyesno("Konfirmasi Hapus", f"Yakin ingin menghapus lagu ID: {self.selected_song_id}?")
            if response:
                if self.logic.library.hapus_lagu_dan_update_playlist(self.selected_song_id, self.logic.users):
                    self.logic._save_library()
                    self.logic._save_users()
                    messagebox.showinfo("Berhasil", "Lagu berhasil dihapus.")
                    self.refresh_content(reset_search=True)
                else:
                    messagebox.showerror("Gagal", "Gagal menghapus lagu. ID tidak valid.")
        else:
            messagebox.showwarning("Perhatian", "Pilih lagu yang akan dihapus.")

    def play_song_admin(self, lagu):
        """Memutar lagu dan memperbarui playback bar."""
        self.logic.playback_queue = PlaybackQueue() # Reset queue
        self.logic.history = PlaybackHistory() # Reset history
        self.logic.play_lagu(lagu)
        self.controller.playback_bar.update_playback_info()
        messagebox.showinfo("Memutar", f"Memutar: {lagu.judul} oleh {lagu.artis}")
        
    def cari_lagu(self):
        """Melakukan pencarian lagu."""
        kunci = self.search_var.get()
        nilai = self.search_entry.get().strip()

        if not nilai:
            messagebox.showwarning("Perhatian", "Masukkan kata kunci pencarian.")
            self.refresh_content(reset_search=True)
            return

        hasil = self.logic.library.cari_lagu(kunci, nilai)
        self.refresh_content(search_results=hasil)
        if not hasil:
            messagebox.showinfo("Pencarian", "Tidak ada lagu yang ditemukan.")

    def logout(self):
        self.selected_song_id = None
        if hasattr(self, 'selected_row'):
             self.selected_row = None
        self.controller.show_frame("LoginPage")


# Halaman Tambah/Update Lagu Admin
class AdminAddUpdateSongPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        self.mode = "tambah" # 'tambah' atau 'update'
        self.lagu_node = None # Hanya digunakan saat update

        self.grid_rowconfigure(0, weight=1)
        self.grid_columnconfigure(0, weight=1)
        
        self.form_frame = ctk.CTkFrame(self, width=450, height=500)
        self.form_frame.grid(row=0, column=0, padx=20, pady=20)
        
        self.title_label = ctk.CTkLabel(self.form_frame, text="", font=ctk.CTkFont(size=20, weight="bold"))
        self.title_label.pack(pady=(20, 10))

        # Form fields
        self.fields = {}
        for text, placeholder in [("Judul:", "Masukkan Judul Lagu"), 
                                 ("Artis:", "Masukkan Artis/Penyanyi"), 
                                 ("Genre:", "Masukkan Genre"), 
                                 ("Album:", "Masukkan Album"), 
                                 ("Durasi:", "m:ss (misal: 4:30)")]:
            frame = ctk.CTkFrame(self.form_frame, fg_color="transparent")
            frame.pack(pady=5, padx=20, fill="x")
            ctk.CTkLabel(frame, text=text, width=80, anchor="w").pack(side="left")
            entry = ctk.CTkEntry(frame, width=300, placeholder_text=placeholder)
            entry.pack(side="right", fill="x", expand=True)
            self.fields[text.replace(":", "")] = entry

        # Field tambahan untuk upload file (simulasi)
        self.upload_frame = ctk.CTkFrame(self.form_frame, fg_color="transparent")
        self.upload_frame.pack(pady=5, padx=20, fill="x")
        ctk.CTkLabel(self.upload_frame, text="File Audio:", width=80, anchor="w").pack(side="left")
        ctk.CTkButton(self.upload_frame, text="Upload File (Simulasi)", command=lambda: messagebox.showinfo("Simulasi", "Upload file audio berhasil...")).pack(side="right", fill="x", expand=True)
        
        # Tombol Aksi
        self.button_frame = ctk.CTkFrame(self.form_frame, fg_color="transparent")
        self.button_frame.pack(pady=20, padx=20)
        
        self.save_button = ctk.CTkButton(self.button_frame, text="SIMPAN PERUBAHAN", command=self.save_lagu)
        self.save_button.pack(side="left", padx=10)
        
        self.cancel_button = ctk.CTkButton(self.button_frame, text="BATAL / KEMBALI", command=self.kembali_ke_admin, fg_color="gray", hover_color="#646464")
        self.cancel_button.pack(side="right", padx=10)

    def refresh_content(self, mode="tambah", lagu_node=None):
        self.mode = mode
        self.lagu_node = lagu_node
        
        # Reset Form
        for entry in self.fields.values():
            entry.delete(0, tk.END)

        if mode == "tambah":
            self.title_label.configure(text="TAMBAH LAGU BARU")
            self.save_button.configure(text="TAMBAH LAGU")
        elif mode == "update" and lagu_node:
            self.title_label.configure(text=f"UPDATE LAGU (ID: {lagu_node.id_lagu})")
            self.save_button.configure(text="SIMPAN PERUBAHAN")
            
            # Isi form dengan data yang sudah ada
            self.fields["Judul"].insert(0, lagu_node.judul)
            self.fields["Artis"].insert(0, lagu_node.artis)
            self.fields["Genre"].insert(0, lagu_node.genre)
            self.fields["Album"].insert(0, lagu_node.album)
            self.fields["Durasi"].insert(0, lagu_node.durasi)

    def save_lagu(self):
        judul = self.fields["Judul"].get().strip()
        artis = self.fields["Artis"].get().strip()
        genre = self.fields["Genre"].get().strip()
        album = self.fields["Album"].get().strip()
        durasi = self.fields["Durasi"].get().strip()

        if not (judul and artis and genre and album and durasi):
            messagebox.showerror("Gagal", "Semua field harus diisi.")
            return

        if self.mode == "tambah":
            lagu = self.logic.library.tambah_lagu(judul, artis, genre, album, durasi)
            self.logic._save_library()
            messagebox.showinfo("Berhasil", f"Lagu '{lagu.judul}' (ID: {lagu.id_lagu}) berhasil ditambahkan!")
        elif self.mode == "update" and self.lagu_node:
            self.lagu_node.judul = judul
            self.lagu_node.artis = artis
            self.lagu_node.genre = genre
            self.lagu_node.album = album
            self.lagu_node.durasi = durasi
            self.logic._save_library()
            messagebox.showinfo("Berhasil", f"Lagu '{judul}' (ID: {self.lagu_node.id_lagu}) berhasil di-update!")

        self.kembali_ke_admin()

    def kembali_ke_admin(self):
        self.controller.show_frame("AdminDashboardPage")


# Dashboard User
class UserDashboardPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent, fg_color="#204060")
        self.controller = controller
        self.logic = logic
        
        self.grid_rowconfigure(1, weight=1) 
        self.grid_columnconfigure(0, weight=1)

        # Header
        header_frame = ctk.CTkFrame(self, fg_color="#4CAF50") # Warna hijau untuk user
        header_frame.grid(row=0, column=0, sticky="ew", padx=0, pady=0)
        header_frame.grid_columnconfigure(0, weight=1)
        
        self.welcome_label = ctk.CTkLabel(header_frame, text="SELAMAT DATANG USER!", font=ctk.CTkFont(size=20, weight="bold"))
        self.welcome_label.grid(row=0, column=0, padx=10, pady=10, sticky="w")
        ctk.CTkButton(header_frame, text="KELUAR", command=self.logout).grid(row=0, column=1, padx=10, pady=10, sticky="e")

        # Konten Utama
        self.content_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.content_frame.grid(row=1, column=0, sticky="nsew", padx=10, pady=10)
        self.content_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)
        self.content_frame.grid_rowconfigure(1, weight=1) # Row 1 untuk artist grid
        
        # 1. Katalog Artis (Simulasi Grid)
        self.artist_label = ctk.CTkLabel(self.content_frame, text="Katalog Artis", font=ctk.CTkFont(size=16, weight="bold"))
        self.artist_label.grid(row=0, column=0, columnspan=4, sticky="w", padx=10, pady=5)
        self.artist_frames = []
        self.create_artist_grid()

        # 2. Menu Pintasan
        menu_frame = ctk.CTkFrame(self.content_frame, fg_color="transparent")
        menu_frame.grid(row=2, column=0, columnspan=4, sticky="ew", pady=(20, 0))
        menu_frame.grid_columnconfigure((0, 1, 2, 3), weight=1)

        ctk.CTkButton(menu_frame, text="Cari Lagu", command=self.open_search).grid(row=0, column=0, padx=10, pady=10, sticky="ew")
        ctk.CTkButton(menu_frame, text="➕ Playlist Baru", command=self.buat_playlist).grid(row=0, column=1, padx=10, pady=10, sticky="ew")
        ctk.CTkButton(menu_frame, text="🎧 Playlist Saya", command=lambda: controller.show_frame("UserPlaylistMenuPage")).grid(row=0, column=2, padx=10, pady=10, sticky="ew")
        ctk.CTkButton(menu_frame, text="Katalog Artis", command=self.show_artist_full_list).grid(row=0, column=3, padx=10, pady=10, sticky="ew")

        # Tombol '+' dan Playlist di bagian bawah (di luar content_frame)
        bottom_menu_frame = ctk.CTkFrame(self, fg_color="transparent")
        bottom_menu_frame.grid(row=2, column=0, sticky="ew", pady=(0, 10))
        bottom_menu_frame.grid_columnconfigure((0, 1), weight=1)
        
        


    def refresh_content(self):
        """Memperbarui nama user dan grid artis."""
        if self.logic.current_user:
            self.welcome_label.configure(text=f"SELAMAT DATANG, {self.logic.current_user.username.upper()}!")
        self.create_artist_grid()
            
    def create_artist_grid(self):
        """Membuat grid simulasi gambar artis."""
        for frame in self.artist_frames:
            frame.destroy()
        self.artist_frames = []

        artis_set = set(lagu.artis for lagu in self.logic.library.get_all_lagu())
        artis_list = sorted(list(artis_set))

        # Tampilkan maksimal 8 artis teratas
        display_artis = artis_list[:8]

        for i, artis in enumerate(display_artis):
            row = i // 4
            col = i % 4
            
            artist_frame = ctk.CTkFrame(self.content_frame, fg_color="transparent", width=150, height=150)
            artist_frame.grid(row=row+1, column=col, padx=10, pady=10, sticky="nsew")
            artist_frame.grid_columnconfigure(0, weight=1)
            self.artist_frames.append(artist_frame)
            
            # Simulasi Gambar (gunakan lingkaran)
            circle = ctk.CTkFrame(artist_frame, width=80, height=80, corner_radius=40, fg_color="#646464")
            circle.pack(pady=5)
            
            ctk.CTkLabel(artist_frame, text=artis).pack()
            
            # Bind untuk aksi klik
            circle.bind("<Button-1>", lambda event, a=artis: self.view_artist_songs(a))
            # Tambahkan ikon untuk memperjelas visualisasi
            ctk.CTkLabel(artist_frame, text="👤", font=ctk.CTkFont(size=30)).place(relx=0.5, rely=0.3, anchor=tk.CENTER) 
            
            # Bind event untuk seluruh frame
            artist_frame.bind("<Button-1>", lambda event, a=artis: self.view_artist_songs(a))

    def view_artist_songs(self, nama_artis):
        """Simulasi membuka halaman list lagu artis."""
        self.open_search(initial_artist=nama_artis)

    def show_artist_full_list(self):
        """Menampilkan menu lengkap katalog artis."""
        artis_set = set(lagu.artis for lagu in self.logic.library.get_all_lagu())
        artis_list = sorted(list(artis_set))
        
        if not artis_list:
            messagebox.showinfo("Katalog Artis", "Library kosong.")
            return

        top = ctk.CTkToplevel(self)
        top.title("Katalog Artis Lengkap")
        ctk.CTkLabel(top, text="Pilih Artis", font=ctk.CTkFont(size=18, weight="bold")).pack(pady=10)
        
        list_frame = ctk.CTkScrollableFrame(top, width=300, height=300)
        list_frame.pack(padx=20, pady=10)
        
        for artis in artis_list:
            ctk.CTkButton(list_frame, text=artis, command=lambda a=artis: [self.view_artist_songs(a), top.destroy()]).pack(pady=5, padx=10, fill="x")

        top.grab_set()

    def buat_playlist(self):
        """Menampilkan dialog untuk membuat playlist baru."""
        if not self.logic.current_user:
            messagebox.showerror("Error", "Anda harus login sebagai user untuk membuat playlist.")
            return

        dialog = ctk.CTkInputDialog(text="Masukkan NAMA Playlist baru:", title="Buat Playlist Baru")
        nama = dialog.get_input()
        
        if nama:
            nama = nama.strip()
            if not nama:
                messagebox.showerror("Gagal", "Nama Playlist tidak boleh kosong.")
                return
            if nama in self.logic.current_user.playlists:
                messagebox.showerror("Gagal", "Nama Playlist sudah ada!")
                return
            
            self.logic.current_user.playlists[nama] = PlaylistLinkedList()
            self.logic._save_users()
            messagebox.showinfo("Berhasil", f"Playlist '{nama}' berhasil dibuat!")

    def open_search(self, initial_artist=None):
        """Membuka halaman pencarian dan tambahkan ke playlist."""
        top = ctk.CTkToplevel(self)
        top.title("Cari Lagu & Tambahkan ke Playlist")
        top.geometry("700x500")
        
        # Frame input pencarian
        search_input_frame = ctk.CTkFrame(top, fg_color="transparent")
        search_input_frame.pack(pady=10)

        # Inisialisasi variabel pencarian
        search_var = tk.StringVar(value="judul")
        search_entry = ctk.CTkEntry(search_input_frame, placeholder_text="Ketik Judul/Artis/Genre/Album", width=250)
        search_entry.pack(side="left", padx=5)
        search_options = ctk.CTkOptionMenu(search_input_frame, values=["judul", "artis", "genre", "album"], variable=search_var)
        search_options.pack(side="left", padx=5)
        ctk.CTkButton(search_input_frame, text="CARI", command=lambda: perform_search(search_entry.get().strip(), search_var.get())).pack(side="left", padx=5)
        
        if initial_artist:
            search_var.set("artis")
            search_entry.insert(0, initial_artist)
        
        # Manual Label untuk Hasil Pencarian
        ctk.CTkLabel(top, text="Hasil Pencarian", font=ctk.CTkFont(weight="bold")).pack(pady=(10, 5))
        results_frame = ctk.CTkScrollableFrame(top, width=600, height=300)
        results_frame.pack(padx=20, pady=(0, 10), fill="x")

        def perform_search(nilai, kunci):
            for widget in results_frame.winfo_children():
                widget.destroy()

            if not nilai:
                ctk.CTkLabel(results_frame, text="Masukkan kata kunci pencarian.", anchor="w").pack(pady=10, padx=5, fill="x")
                return

            hasil = self.logic.library.cari_lagu(kunci, nilai)

            # Header Tabel
            headers = ["No", "ID", "Judul", "Artis", "Genre", "Album", "Durasi", "Aksi"]
            for i, text in enumerate(headers):
                results_frame.grid_columnconfigure(i, weight=1)
                ctk.CTkLabel(results_frame, text=text, font=ctk.CTkFont(weight="bold")).grid(row=0, column=i, sticky="w", padx=5)

            if not hasil:
                ctk.CTkLabel(results_frame, text="[Tidak ada lagu yang ditemukan.]", anchor="w").grid(row=1, column=0, columnspan=8, padx=5, pady=10)
                return

            for i, lagu in enumerate(hasil, 1):
                row_frame = ctk.CTkFrame(results_frame, fg_color="transparent")
                row_frame.grid(row=i, column=0, columnspan=8, sticky="ew")
                row_frame.grid_columnconfigure((0, 1, 2, 3, 4, 5, 6, 7), weight=1)

                ctk.CTkLabel(row_frame, text=f"{i}").grid(row=0, column=0, sticky="w", padx=5)
                ctk.CTkLabel(row_frame, text=lagu.id_lagu).grid(row=0, column=1, sticky="w", padx=5)
                ctk.CTkLabel(row_frame, text=lagu.judul[:20]).grid(row=0, column=2, sticky="w", padx=5)
                ctk.CTkLabel(row_frame, text=lagu.artis[:15]).grid(row=0, column=3, sticky="w", padx=5)
                ctk.CTkLabel(row_frame, text=lagu.genre[:10]).grid(row=0, column=4, sticky="w", padx=5)
                ctk.CTkLabel(row_frame, text=lagu.album[:13]).grid(row=0, column=5, sticky="w", padx=5)
                ctk.CTkLabel(row_frame, text=lagu.durasi).grid(row=0, column=6, sticky="w", padx=5)
                
                # Tombol Aksi: Play dan Tambah ke Playlist
                action_frame = ctk.CTkFrame(row_frame, fg_color="transparent")
                action_frame.grid(row=0, column=7, sticky="w", padx=5)
                
                ctk.CTkButton(action_frame, text="▶️", width=30, command=lambda l=lagu: self.play_song_user(l)).pack(side="left", padx=2)
                ctk.CTkButton(action_frame, text="➕", width=30, command=lambda l=lagu: self.add_to_playlist_dialog(l)).pack(side="left", padx=2)
                
        # Panggil pencarian otomatis jika ada initial_artist
        if initial_artist:
            perform_search(initial_artist, "artis")
        
        top.grab_set()

    def add_to_playlist_dialog(self, lagu):
        """Dialog untuk memilih playlist tujuan."""
        if not self.logic.current_user.playlists:
            messagebox.showwarning("Perhatian", "Anda belum memiliki Playlist. Buat Playlist baru terlebih dahulu.")
            return

        top = ctk.CTkToplevel(self)
        top.title(f"Tambahkan '{lagu.judul}'")
        
        ctk.CTkLabel(top, text="Pilih Playlist Tujuan:", font=ctk.CTkFont(size=16, weight="bold")).pack(pady=10)
        
        playlist_names = list(self.logic.current_user.playlists.keys())
        
        def add_song(name):
            playlist = self.logic.current_user.playlists[name]
            # Cek duplikat (opsional)
            if lagu.id_lagu in playlist.to_list_id():
                messagebox.showwarning("Perhatian", f"Lagu '{lagu.judul}' sudah ada di Playlist '{name}'.")
                top.destroy()
                return

            playlist.tambah_lagu(lagu)
            self.logic._save_users()
            messagebox.showinfo("Berhasil", f"Lagu '{lagu.judul}' ditambahkan ke Playlist '{name}'!")
            top.destroy()
            
        for name in playlist_names:
            ctk.CTkButton(top, text=name, command=lambda n=name: add_song(n)).pack(pady=5, padx=20, fill="x")

        ctk.CTkButton(top, text="Batal", command=top.destroy, fg_color="gray", hover_color="#646464").pack(pady=10, padx=20, fill="x")
        top.grab_set()
        
    def play_song_user(self, lagu):
        """Memutar lagu dan memperbarui playback bar."""
        self.logic.playback_queue = PlaybackQueue() # Reset queue
        self.logic.history = PlaybackHistory() # Reset history
        self.logic.play_lagu(lagu)
        self.controller.playback_bar.update_playback_info()
        messagebox.showinfo("Memutar", f"Memutar: {lagu.judul} oleh {lagu.artis}")

    def logout(self):
        self.logic.current_user = None
        self.logic.current_playing = None
        self.logic.playback_queue = PlaybackQueue()
        self.logic.history = PlaybackHistory()
        self.controller.show_frame("UserSignupPage") # Kembali ke Menu Login/Signup User


# Menu Playlist User
class UserPlaylistMenuPage(ctk.CTkFrame):
    def __init__(self, parent, controller, logic):
        super().__init__(parent)
        self.controller = controller
        self.logic = logic
        
        self.grid_rowconfigure(1, weight=1) 
        self.grid_columnconfigure(0, weight=1)
        
        ctk.CTkLabel(self, text="PLAYLIST SAYA", font=ctk.CTkFont(size=24, weight="bold")).grid(row=0, column=0, pady=10)
        
        self.main_frame = ctk.CTkFrame(self, fg_color="transparent")
        self.main_frame.grid(row=1, column=0, sticky="nsew", padx=20, pady=10)
        self.main_frame.grid_columnconfigure(0, weight=1)
        self.main_frame.grid_rowconfigure(0, weight=1)
        
        # Konten Frame (akan diisi oleh daftar playlist atau lagu)
        self.content_frame = ctk.CTkFrame(self.main_frame)
        self.content_frame.grid(row=0, column=0, sticky="nsew")
        
        self.kembali_button = ctk.CTkButton(self, text="< KEMBALI KE DASHBOARD", command=lambda: controller.show_frame("UserDashboardPage"))
        self.kembali_button.grid(row=2, column=0, pady=10)
        
        self.current_view = "menu" # 'menu' (daftar playlist) atau 'detail' (list lagu playlist)
        self.current_playlist_name = None

    def refresh_content(self, view="menu", playlist_name=None):
        """Memuat ulang tampilan menu/detail playlist."""
        if not self.logic.current_user:
            # Perlu diperiksa jika pengguna tidak sengaja mencapai halaman ini tanpa login
            self.controller.show_frame("UserSignupPage")
            messagebox.showerror("Error", "Sesi tidak ditemukan. Silakan login kembali.")
            return

        self.current_view = view
        self.current_playlist_name = playlist_name

        for widget in self.content_frame.winfo_children():
            widget.destroy()
            
        self.content_frame.grid_columnconfigure(0, weight=1)

        if view == "menu":
            self.display_playlist_menu()
        elif view == "detail" and playlist_name:
            self.display_playlist_detail(playlist_name)
        else:
            self.display_playlist_menu()

    def display_playlist_menu(self):
        """Menampilkan daftar semua playlist."""
        ctk.CTkLabel(self.content_frame, text="Daftar Playlist Anda:", font=ctk.CTkFont(weight="bold")).pack(pady=10)
        
        if not self.logic.current_user.playlists:
            ctk.CTkLabel(self.content_frame, text="[Anda belum memiliki playlist.]").pack(pady=20)
            return
            
        for name in self.logic.current_user.playlists.keys():
            frame = ctk.CTkFrame(self.content_frame, fg_color="transparent")
            frame.pack(pady=5, padx=20, fill="x")
            
            ctk.CTkButton(frame, text=name, command=lambda n=name: self.refresh_content("detail", n), anchor="w").pack(side="left", fill="x", expand=True)
            ctk.CTkButton(frame, text="🗑️", width=30, fg_color="red", hover_color="#800000", command=lambda n=name: self.delete_playlist(n)).pack(side="right", padx=5)

    def display_playlist_detail(self, nama_playlist):
        """Menampilkan detail (list lagu) dari playlist yang dipilih."""
        playlist = self.logic.current_user.playlists.get(nama_playlist)
        if not playlist:
            messagebox.showerror("Error", "Playlist tidak ditemukan.")
            self.refresh_content("menu")
            return
            
        # Manual Label untuk Playlist Detail 
        ctk.CTkLabel(self.content_frame, text=f"Isi Playlist: {nama_playlist}", font=ctk.CTkFont(weight="bold")).pack(pady=(10, 5))
        
        scroll_frame = ctk.CTkScrollableFrame(self.content_frame) 
        scroll_frame.pack(fill="both", expand=True, padx=10, pady=(0, 10))
        scroll_frame.grid_columnconfigure((0, 1, 2, 3, 4, 5), weight=1)

        ctk.CTkButton(self.content_frame, text="< Kembali ke Daftar Playlist", command=lambda: self.refresh_content("menu")).pack(pady=10)

        lagu_list = playlist.to_list_lagu()
        
        # Header Tabel
        headers = ["No", "ID", "Judul", "Artis", "Genre", "Durasi", "Aksi"]
        for i, text in enumerate(headers):
            scroll_frame.grid_columnconfigure(i, weight=1)
            ctk.CTkLabel(scroll_frame, text=text, font=ctk.CTkFont(weight="bold")).grid(row=0, column=i, sticky="w", padx=5)

        if not lagu_list:
            ctk.CTkLabel(scroll_frame, text="[Playlist kosong.]", anchor="w").grid(row=1, column=0, columnspan=7, padx=5, pady=10)
            return
        
        for i, lagu in enumerate(lagu_list, 1):
            row_frame = ctk.CTkFrame(scroll_frame, fg_color="transparent")
            row_frame.grid(row=i, column=0, columnspan=7, sticky="ew")
            row_frame.grid_columnconfigure((0, 1, 2, 3, 4, 5, 6), weight=1)
            
            ctk.CTkLabel(row_frame, text=f"{i}").grid(row=0, column=0, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.id_lagu).grid(row=0, column=1, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.judul[:20]).grid(row=0, column=2, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.artis[:15]).grid(row=0, column=3, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.genre[:10]).grid(row=0, column=4, sticky="w", padx=5)
            ctk.CTkLabel(row_frame, text=lagu.durasi).grid(row=0, column=5, sticky="w", padx=5)
            
            # Tombol Play & Hapus
            action_frame = ctk.CTkFrame(row_frame, fg_color="transparent")
            action_frame.grid(row=0, column=6, sticky="w", padx=5)
            
            ctk.CTkButton(action_frame, text="▶️", width=30, command=lambda l=lagu, p=playlist: self.play_song_from_playlist(l, p)).pack(side="left", padx=2)
            ctk.CTkButton(action_frame, text="❌", width=30, fg_color="red", hover_color="#800000", command=lambda l=lagu, n=nama_playlist: self.hapus_lagu_dari_playlist(l.id_lagu, n)).pack(side="left", padx=2)

    def delete_playlist(self, nama_playlist):
        response = messagebox.askyesno("Konfirmasi Hapus", f"Yakin ingin menghapus Playlist '{nama_playlist}'?")
        if response:
            if nama_playlist in self.logic.current_user.playlists:
                del self.logic.current_user.playlists[nama_playlist]
                self.logic._save_users()
                messagebox.showinfo("Berhasil", f"Playlist '{nama_playlist}' berhasil dihapus.")
                self.refresh_content("menu")
            else:
                messagebox.showerror("Gagal", "Playlist tidak ditemukan.")

    def hapus_lagu_dari_playlist(self, id_lagu, nama_playlist):
        playlist = self.logic.current_user.playlists.get(nama_playlist)
        if not playlist:
            messagebox.showerror("Error", "Playlist tidak ditemukan.")
            self.refresh_content("menu")
            return
            
        response = messagebox.askyesno("Konfirmasi Hapus", f"Yakin ingin menghapus lagu ID: {id_lagu} dari Playlist '{nama_playlist}'?")
        if response:
            if playlist.hapus_lagu_berdasarkan_id(id_lagu):
                self.logic._save_users()
                messagebox.showinfo("Berhasil", "Lagu berhasil dihapus dari Playlist.")
                self.refresh_content("detail", nama_playlist)
            else:
                messagebox.showerror("Gagal", "Lagu tidak ditemukan di Playlist.")
                
    def play_song_from_playlist(self, lagu_mulai, playlist):
        """Memutar lagu dari playlist, mengisi queue dengan lagu berikutnya."""
        
        # 1. Reset Queue & History
        self.logic.playback_queue = PlaybackQueue() 
        self.logic.history = PlaybackHistory()
        
        # 2. Isi Queue dengan lagu-lagu berikutnya di playlist
        curr = playlist.head
        start_queue = False
        while curr:
            if curr.lagu.id_lagu == lagu_mulai.id_lagu:
                start_queue = True
                curr = curr.next
                continue
            
            if start_queue:
                self.logic.playback_queue.enqueue(curr.lagu)
            
            curr = curr.next

        # 3. Putar lagu yang dipilih
        self.logic.play_lagu(lagu_mulai)
        self.controller.playback_bar.update_playback_info()
        messagebox.showinfo("Memutar Playlist", f"Memutar: {lagu_mulai.judul} dari Playlist '{self.current_playlist_name}'. Queue diisi.")


# ------------------------
# EKSEKUSI APLIKASI
# ------------------------
if __name__ == "__main__":
    # Inisialisasi Logika/Backend
    app_logic = SyfoplayAppLogic()
    
    # Inisialisasi GUI
    app_gui = SyfoplayGUI(app_logic)
    app_gui.mainloop()

Data tersimpan sebelum keluar.
